# 自回归分布滞后模型

## 原理讲解

在自回归模型中，也可以引入其他变量来构成“自回归分布滞后模型”（Autoregressive Distributed Lag Model，简记 $ADL(p,q)$）：

$$y_{t}=\beta_{0}+\beta_{1} y_{t-1}+\cdots+\beta_{p} y_{t-p}+\gamma_{1} x_{t-1}+\cdots+\gamma_{q} x_{t-q}+\varepsilon_{t}\tag{1}$$

如果自回归分布滞后模型满足以下假定则万事大吉，可以用OLS来估计它。
1. $\mathrm{E}\left(\varepsilon_{t} | y_{t-1}, y_{t-2}, \cdots, x_{1, t-1}, x_{1, t-2}, \cdots, x_{K, t-1}, x_{K, t-2}, \cdots\right)=0$。这个假定类似于严格外生性假设，它意味着扰动项 $\varepsilon_{t}$ 与所有解释变量的整个历史全部无关。这保证了对滞后期数 $\left(p, q_{1}, \cdots, q_{K}\right)$ 的设定是正确的。如果滞后期数的设定不正确，比如，真实模型还应该包括 $y_{t-（p+1）}$，但该项 $\beta_{p+1} y_{t-(p+1)}$ 却被纳入扰动项中，则扰动项 $\varepsilon_{t}$ 便与解释变量相关，导致OLS不致。
2. $\left\{y_{t}, x_{1 t}, \cdots, x_{K t}\right\}$ 为渐近独立的平稳序列。
3. $\left\{y_{t}, x_{1 t}, \cdots, x_{K t}\right\}$ 有非零的有限四阶矩。
4. 解释变量无完全多重共线性。

对滞后期数的选择可以使用信息准则（最小化 AIC 或 BIC），或使用 $t,F$ 检验来检验最后期系数的显著性。更一般地，可以在 $ARMA$ 模型中引入其他变量，称为“$ARMAX$”模型。

## statsmodels 库实现

In [ ]:
def ADLxx(Y, X, p, q):
    """
    待估计方程：y = c + y(-1) +....+y(-p) + x(-1) + ... + x(-q)
    获取自回归分布滞后模型的估计向量
    X  - 解释变量，列向量
    Y  - 被解释变量，列向量
    p  - ADL 模型 Y 的滞后阶数,标量
    q  - ADL 模型 X 的滞后阶数,标量
    """
    T = len(Y)
    N = max(p,q)+1

    ADLy = Y(N:T);
    c = ones(length(ADLy),1);  %常数项
    ADLx = zeros(length(ADLy),1+p+q);
    ADLx(:,1) = c;
    for i=1:p, ADLx(:,1+i) = Y(N-i:T-i);end
    for i=1:q, ADLx(:,1+p+i) = X(N-i:T-i);end
    

## matlab 实现

ADLxx.m
```matlab
function [ADLy,ADLx] = ADLxx(Y,X,p,q)

%待估计方程：y = c + y(-1) +....+y(-p) + x(-1) + ... + x(-q)
% 获取自回归分布滞后模型的估计向量
%  X   - 解释变量，列向量
%  Y   - 被解释变量，列向量
%  p   - ADL模型Y的滞后阶数,标量
%  q   - ADL模型X的滞后阶数,标量

T = length(Y);
N = max(p,q)+1;

ADLy = Y(N:T);
c = ones(length(ADLy),1);  %常数项
ADLx = zeros(length(ADLy),1+p+q);
ADLx(:,1) = c;
for i=1:p, ADLx(:,1+i) = Y(N-i:T-i);end
for i=1:q, ADLx(:,1+p+i) = X(N-i:T-i);end

% % 1.载入数据
% data = xlsread('C:\Users\Administrator\Desktop\hourse.xlsx');
% f1 = data(:,2); f2 = data(:,3); e = data(:,6);
% 
% % 2.估计ADL模型f1 = c + f1(-1) + f1(-2) + e(-1)
% [ADLy,ADLx] = ADLxx(f1,e,2,1);
% beta = regress(ADLy,ADLx);
```